In [ ]:
#On importe le module os qui fournit une façon portable d'utiliser les fonctionnalités dépendantes du système d'exploitation.
import os

#On spécifie notre réperoite de travil ou se trouve les données.
base_dir = '../input/dogs-vs-cats'

#On spécifie le chemin d'accès des fichiers zip : train et test1
train_dir = os.path.join(base_dir, 'train.zip')
test_dir = os.path.join(base_dir, 'test1.zip')

In [ ]:
# On Importe le module zipfile qui permet de manipuler des fichiers ZIP.
import zipfile

#On extrait nos deux fichiers zip:

#Fichier train.zip:
with zipfile.ZipFile(train_dir,'r') as z:
    z.extractall()

#Fichier test1.zip:
with zipfile.ZipFile(test_dir,'r') as z:
    z.extractall()

#Après l'extraction des données, on obtient deux dossiers stockés dans l'output.

In [ ]:
#On crée une liste "pics" qui contient le nom de tous les fichiers dans le dossier "train"
images = os.listdir('./train')

#On importe la bibliothèque pandas permettant la manipulation et l'analyse des données.
import pandas as pd

#On utilise la fonction DataFrame() qui organise les données en lignes et en colonnes:
data = pd.DataFrame(images)

#On change le nom de la colonne de "0" à "image":
data = data.rename(columns = {0:'image'})

#On ajoute "./train/" avant le nom de chaque element dans la colonne "image":
data['image'] = data['image'].apply(lambda x: './train/' + x)

#Notons que la colonne "image" contient le chemin d'accès de chaque image !

#On crée une nouvelle variable "label" qui prend les valeurs "dog" ou "cat" selon x
data['label'] = data['image'].apply(lambda x: 'cat' if 'cat' in x else 'dog')

#On observe les 5 premières lignes de notre data:
data.head()

In [ ]:
data.shape
#Notre data contient 25000 observations (lignes) et 2 variables (colonnes)

In [ ]:
#On scinde notre data en trois sous-ensembles de données:

#Données d'entraînement : 80% de lignes (20000) et 2 colonnes
train_data = data.iloc[0:20000, 0:2]
print(train_data.shape)

#Données de validation: 10% de lignes (2500) et 2 colonnes
validation_data = data.iloc[20000:22500, 0:2]
print(validation_data.shape)

#Données de test: 10% de lignes (2500) et 2 colonnes:
test_data = data.iloc[22500:25000, 0:2]
print(test_data.shape)

In [ ]:
# On importe le module ImageDataGenerator qui génère des lots de données d'images tensorielles avec une augmentation des données en temps réel.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Comme 255 est la valeur maximale d'un pixel, on change l'echelle de ces valeurs en divisant par 255 afin qu'elles prennent de nouvelles valeurs allant de 0 à 1.
train_datagen =  ImageDataGenerator(rescale=1/255)
validation_datagen =  ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
#Transformer nos dataframes en batches d'images uniformisées:

train_genarator = train_datagen.flow_from_dataframe(train_data, target_size=(150,150),
                                                    x_col='image',y_col='label',class_mode='binary',batch_size = 32)

validation_genarator = train_datagen.flow_from_dataframe(validation_data,target_size=(150,150),
                                                         x_col='image',y_col='label',class_mode='binary',batch_size= 32)

test_generator = train_datagen.flow_from_dataframe(test_data,target_size=(150,150),
                                                         x_col='image',y_col='label',class_mode='binary',batch_size= 32)

In [ ]:
#Importer tous les bibliothèques et les modules nécéssaire à la construction de notre modèle:

import tensorflow as tf 

from tensorflow import keras

from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras import regularizers

#from keras.preprocessing import image

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from keras import optimizers

#from sklearn.model_selection import train_test_split

#from tensorflow.keras.utils import to_categorical

from tqdm import tqdm

from keras.models import load_model

In [ ]:
#On spécifie un modèle sequentiel : empilement simple de couches où chaque couche a exactement un tenseur d'entrée et un de sortie
model = Sequential()

#On définit trois couches de convolution, chacune suivie d'une couche de maxpooling de dimension 2x2

#La première couche contient 32 filtres, de dimension 3x3 et avec la fonction d'activation "relu". 
#Notons qu'elle prenne des élements d'entrée de dimension 150x150x3.
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))

#En passant à la deuxième couche, on a augmenté le nombre des filtres à 64.
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))

#Et finalement, la dernière couche contient 128 filtres. 
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))

#On utilise la fonction Flatten pour applatir les tenseurs d'entrée multidimensionnels en une seule dimension.
model.add(Flatten())

#La couche Dense transforme la matrice d'entrée en vecteur ligne, elle performe aussi des opérations de simplification.
model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.01)))

#La couche Dropout donne de manière aléatoire la valeur 0 aux elements d'entrée avec une fréquence de 0.25 à chaque étape pendant la durée de l'entrainement.
model.add(Dropout(0.25))

#Cette couche dense génère un seul nombre et utilise la fonction d'activation "sigmoid".
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#Un aperçu syntéthique de notre modèle et ses couches:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
#On utilise l'optimisateur Adam avec un pas d'apprentissage de 0.0001: 
opt = keras.optimizers.Adam(learning_rate=0.0001)

#On configure le modèle pour l'entrainement: la fontion loss, l'optimisateur et la fonction metrics permettant d'évaluer le modèle.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Un résumé utile du modèle, qui comprend : Le nom et le type de toutes les couches du modèle. Forme de sortie pour chaque couche.
model.summary()
#On remarque une réduction de l'image (dimension de la matrice) et augmentation de la profondeur de la matrice.

In [ ]:
#Pour libérer de la mémoire et accélérer le processus:
import gc
gc.collect()

In [ ]:
#On spécifie la mesure de performance à monitorer, le déclencheur, et une fois déclenché, il arrêtera le processus d'entrainement.

#Aussi, On réduit le pas d'apprentissage lorsqu'une métrique a cessé de s'améliorer.

cb = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=5, 
                                              restore_best_weights=True),
             tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss' , patience=3)]

#Une fois le modèle spécifié, on commence l'entrainement avec 100 époque:
history = model.fit(train_genarator, epochs = 30, validation_data= validation_genarator,
                    callbacks = cb)

In [ ]:
#On génère les predictions du modèle par rapport aux données de test:
predict = model.predict_generator(test_generator)

predict = pd.DataFrame(predict)

predict = predict.rename(columns = {0:'Cat probability'})

predict['Dog probability'] = 1 - predict['Cat probability']

display(predict)

In [ ]:
#Importation des modules:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

#Si la probabilité dépasse 0.5, la prédiction renvoie "cat":
predictions = (model.predict(test_generator) >= 0.5).astype(np.int)

#On génère la matrice de confusion fournissant une comparaison de la classes prédite et la classe actuelle:
cm = confusion_matrix(test_generator.labels, predictions, labels=[0, 1])

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=[0.5, 1.5], labels=["cat", "dog"])
plt.yticks(ticks=[0.5, 1.5], labels=["cat", "dog"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
#Rapport de classification:
clr = classification_report(test_generator.labels, predictions, labels=[0, 1], target_names=["cat", "dog"])
print(clr)

In [ ]:
#On crée un répértoire de travail où on va enregistrer notre modèle
os.makedirs("./Saved_Models")

#Ensuite, on enregistre le modèle 
from keras.models import load_model
model.save('./Saved_Models/model_cat_dog.h5')

In [ ]:
#On observe l'évolution des valeurs de perte et de précision selon les époques :
history_df = pd.DataFrame(history.history)
print(history_df)

In [ ]:
#Le graphique de la fonction de perte:
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
#Le graphique de la fonction de précision:
history_df.loc[:, ['accuracy', 'val_accuracy']].plot();
print("Maximum validation accuracy : {}".format(history_df['val_accuracy'].max()))